In [24]:
import os
from dotenv import load_dotenv

MONGO_DB_URI = os.getenv("MONGO_DB_URI")

In [25]:
# %pip install llama-index-vector-stores-mongodb

In [26]:
import HFH

In [27]:
import pymongo
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.core import StorageContext

mongodb_client = pymongo.MongoClient(MONGO_DB_URI)
atlas_vector_search = MongoDBAtlasVectorSearch(
    mongodb_client,
    db_name = "llamaindex_db",
    collection_name = "test",
    index_name = "vector_index"
)
storage_context = StorageContext.from_defaults(vector_store=atlas_vector_search)

In [1]:
####  Create embedding and store into database

# from llama_index.core import SimpleDirectoryReader
# from llama_index.core import VectorStoreIndex,load_index_from_storage


# docs = SimpleDirectoryReader("./data/test/").load_data()
# print(docs)
# vector_store_index = VectorStoreIndex.from_documents(
#     documents=docs, storage_context=storage_context, show_progress=True
# )

In [ ]:
#### Load embedding from database and create query retriver

from llama_index.core import VectorStoreIndex
index = VectorStoreIndex.from_vector_store(atlas_vector_search)

In [30]:
retriever = index.as_retriever(similarity_top_k=3)
nodes = retriever.retrieve("what is iris?")

for node in nodes:
    print(node)

Batches: 100%|██████████| 1/1 [00:00<00:00, 28.40it/s]

Node ID: 99a28f82-89a5-45f6-9659-be65cc47fefc
Text: IRIS Iris kemaonensis
Iridaceae This wild flower has one of the loveliest names of all
flowers. The iris is the Greek name for the rainbow; the flowers have
many rainbow hues, as the name suggests. Sometimes women are named
after the flower . The iris ...
Score:  0.923

Node ID: 70e83bef-9cf5-476f-aa37-d0021eb7e0e5
Text: LOTUS Nelumbo nucifera The lotus, known as the Kamal, or Padma,
is one of our best known wild flowers. This beautiful flower mostly
pink and sometimes white, is found all over the warmer parts of India
and grows in lakes, ponds and rivers. The big flowers have a pleasant
fragrance and bloom from August to October . The leaves are large and
round...
Score:  0.892

Node ID: fa0fab03-459b-4711-9da2-c3ff6612014d
Text: GLOSSAR Y • achene a small dry one-seeded fruit that is usually
one of many in a fruiting head • anther the port of the stamen which
contains the pollen • axil the angle between the upper side of the
leaf an

In [50]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import PromptTemplate

# Instantiate Atlas Vector Search as a retriever
vector_store_retriever = VectorIndexRetriever(index=index, similarity_top_k=5)

query_engine = index.as_query_engine(
    text_qa_template= PromptTemplate("""
        Below is an instruction that describes a task.     
        Write a response that appropriately completes the request.\n\n    
        ### Instruction:\n{query_str}\n\n### Response:""")
)

In [51]:
query_engine.query("which flowers are found below 2200m height")


Batches: 100%|██████████| 1/1 [00:00<00:00, 24.19it/s]


ValueError: Calculated available context size -8033 was not non-negative.

In [ ]:
# print("\nSource documents: ")
# print(response.source_nodes)